# Interactive Chemostat Model  

This notebook explores a **single‑species chemostat** with Monod growth kinetics. Use the sliders to vary parameters and see how the system either *reaches a steady biomass* (growth balances dilution) or is *washed out* (biomass $\to0$).

---

## 1&nbsp;&nbsp;Dimensional model  

Nutrient concentration $c(t)$ and biomass density $\rho(t)$ are governed by the following equations:

$$
\begin{aligned}
\frac{dc}{dt} &= \frac{Q}{V}\bigl(c_{\text{in}}-c\bigr)
                 -\nu\,\rho\,
                 \mu_{\max}\frac{c}{K+c},\\[6pt]
\frac{d\rho}{dt} &= \rho\!\left[
    \mu_{\max}\frac{c}{K+c}-\frac{Q}{V}\right].
\end{aligned}
$$

* $Q$ — flow rate in and out of the chemostat
* $V$ — vessel volume  
* $c_{\text{in}}$ — nutrient concentration in the input media  
* $K$ — Monod half‑saturation constant  
* $\mu_{\max}$ — maximal specific growth rate  
* $\nu$ — yield (set to 1 for this example).

Chemostats are widely used in experimental microbiology to maintain microbes in **continuous exponential growth** under a controlled nutrient limitation. Understanding the parameter regime that avoids *wash‑out* is essential for designing such experiments and for interpreting **open natural systems** (e.g. gut, soil pores, aquatic environments) that operate similarly to chemostats.

---

## 2&nbsp;&nbsp;Dimensionless form  

Often, one can reduce the number of effective parameters of a model by **non-dimensionalization**. In this case, we can eliminate parameters by rescaling concentrations by $K$ and time by the *residence time*
$T=V/Q$:

$$
\dot{\tilde c}= \tilde c_{\text{in}}-\tilde c-
\tilde\rho\,\gamma\frac{\tilde c}{1+\tilde c},
\qquad
\dot{\tilde\rho}= \tilde\rho
\left[\gamma\frac{\tilde c}{1+\tilde c}-1\right],
$$

where  

$$
\gamma=\mu_{\max}T, \qquad
\tilde c_{\text{in}}=\frac{c_{\text{in}}}{K}.
$$

* $\tilde c_{\text{in}}$ measures **how concentrated the feed is** relative to $K$.
* $\gamma$ compares **potential growth** to **dilution**.

Thus, only these two effective parameters control the qualitative dynamics.

---

## 3&nbsp;&nbsp;Wash‑out condition  

Because $\tilde\rho$ is multiplied in its own equation, biomass goes
extinct unless **net growth ≥ dilution**:

$$
\boxed{\;\gamma > 1+\frac{1}{\tilde c_{\text{in}}}\;}
\quad\Longrightarrow\quad \text{persistence.}
$$

If the inequality fails, the stable fixed point is $(\tilde c,\tilde\rho)=(\tilde c_{\text{in}},0)$:
nutrient stays at the feed value and biomass is washed out.

The diagram on the right marks this boundary and shows your current
$(\tilde c_{\text{in}},\gamma)$.

---

## 4&nbsp;&nbsp;Interactive exploration  

Move the sliders to simulate the chemostat under different parameters:

* **Time‑series (left):** dynamics of biomass and nutrient concentration over time.  
* **Parameter space (right):** phage diagram of $(\tilde c_{\text{in}},\gamma)$ showing the washout boundary. Red shading indicates wash‑out, green shading denotes persistence. Red dot represents position of current chemostat parameters. 



In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as wd

plt.rcParams["figure.dpi"] = 300

# ODE system (dimensionless)
def rhs(t, y, c_in_tilde, gamma):
    c, rho = y
    dc = c_in_tilde - c - rho * gamma * c/(1+c)
    drho = rho * (gamma * c/(1+c) - 1)
    return [dc, drho]

def simulate(mu_max=1.0, Q=0.3, c_in=10.0,
             K=1.0, V=1.0, t_end=30):
    gamma = mu_max * V / Q
    c_in_tilde = c_in / K
    washout = gamma <= 1 + 1/c_in_tilde

    # integrate
    y0 = [0.8*c_in_tilde, 0.05]
    sol = solve_ivp(rhs, (0, t_end), y0,
                    args=(c_in_tilde, gamma), dense_output=True,
                    max_step=0.1)
    t = np.linspace(0, t_end, 400)
    c, rho = sol.sol(t)

    # plotting
    fig, (ax_ts, ax_par) = plt.subplots(1,2, figsize=(11,4))

    # --- time series
    ax_ts.plot(t, c, label=r'$\tilde c$', color='tab:blue')
    ax_ts.plot(t, rho, label=r'$\tilde\rho$', color='tab:orange')
    ax_ts.set_xlabel('t / T'); ax_ts.set_ylabel('dimensionless conc.')
    ax_ts.set_title('Wash‑out' if washout else 'Persistence')
    ax_ts.legend()

    # --- parameter space
    x = np.linspace(0.1, max(5, c_in_tilde*1.3), 400)
    y_bound = 1 + 1/x
    ax_par.plot(x, y_bound, 'k--', label='Wash‑out boundary')
    ax_par.fill_between(x, 0, y_bound, alpha=0.25, color='mistyrose')
    ax_par.fill_between(x, y_bound, y_bound.max()*1.1,
                        alpha=0.25, color='honeydew')
    ax_par.scatter([c_in_tilde], [gamma], color='red',
                   zorder=5, label='Current point')
    ax_par.set_xlabel(r'$\tilde c_{\mathrm{in}}$')
    ax_par.set_ylabel(r'$\gamma$')
    ax_par.set_xlim(0, x.max())
    ax_par.set_ylim(0, max(gamma*1.2, y_bound.max()*1.05))
    ax_par.set_title('Wash‑out map')
    ax_par.legend(fontsize=7, loc='upper right')

    plt.tight_layout()
    plt.show()

ui = wd.interactive(
    simulate,
    mu_max = wd.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.05,
                            description=r'$\mu_{\max}$ (h⁻¹)'),
    Q      = wd.FloatSlider(value=0.3, min=0.05, max=1.0, step=0.05,
                            description='$Q$ (L h⁻¹)'),
    c_in   = wd.FloatSlider(value=10.0, min=0.5, max=20.0, step=0.5,
                            description='$c_{in}$ (mM)'),
    K   = wd.FloatSlider(value=1, min=0.1, max=10.0, step=0.05,
                            description='$K$ (mM)'),
    V   = wd.FloatSlider(value=1.5, min=0.1, max=7.0, step=0.05,
                            description='$V$ (L)'),
    t_end   = wd.FloatSlider(value=30, min=1, max=100.0, step=1,
                            description='$t_{end}$ (h)')
)
display(ui)

interactive(children=(FloatSlider(value=1.0, description='$\\mu_{\\max}$ (h⁻¹)', max=2.0, min=0.1, step=0.05),…

---

## 5&nbsp;&nbsp; Why study chemostat models?

* **Experimental design:** When operating a chemostat you choose $Q$,
  $c_{\text{in}}$, and temperature (affecting $\mu_{\max}$).  The wash‑out
  inequality tells you *where* continuous growth is possible.

* **Ecological insights:** Some natural micro‑habitats behave like
  chemostats.  For example, river biofilms experience constant flow.  
  $\gamma$ and $\tilde c_{\text{in}}$ provide a compact way to compare
  such habitats and predict whether resident microbes can persist.

* **Mathematical clarity:** Reducing the model to two
  dimensionless numbers isolates *mechanism* (growth vs. dilution)
  from units and experimental details.
